In [7]:
import math
import torch
from gamesopt.train_distributed import train, BaseConfig, BaseData
from gamesopt.games import GameType
from gamesopt.optimizer.base import OptimizerType
from gamesopt.aggregator import AggregatorType
from gamesopt.attacks import AttackType
from gamesopt.games.utils import create_matrix
from dataclasses import dataclass

# %matplotlib widget
%matplotlib inline
%load_ext autoreload
%autoreload 1

In [9]:
import os
# os.environ['MLFLOW_TRACKING_URI'] += os.getcwd().split('/')[-1]
os.getcwd().split('/')

['', 'home', 'nazya', 'sgda-ra']

In [10]:
import os
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_EXPERIMENT_NAME'] = 'Default'
os.environ['MLFLOW_TRACKING_URI'] = '/home/nazya/mlruns/' + os.getcwd().split('/')[-1]
# os.system('mlflow ui --backend-store-uri /home/nazya/mlruns')

In [17]:
config = {
    'n_iter': 6001,
    'n_peers': 20,
    'n_byzan': 4,
    'game_type': GameType.QUADRATIC,
    'num_samples': 1000,
    'dim': 16,
    'with_bias': True,
    'mu': 0.1,
    'ell': 1000.0,
    'attack_type': AttackType.ALIE,
    'n_attacking': 1,
    'ipm_epsilon': 0.1,
    'rn_sigma': 10.0,
    'alie_z': 100.0,
    'use_bucketing': True,
    'bucketing_s': 2,
    'aggregator_type': AggregatorType.RFA,
    'trimmed_mean_b': 10,
    'krum_m': 2,
    'clipping_tau': 10,
    'clipping_n_iter': 3,
    'rfa_T': 10,
    'rfa_nu': 0.1,
    'optimizer_type': OptimizerType.SGDARA,
    'alpha': 0.1,
    'lr': 3.3333333333333335e-05,
    'lr_inner': 3.3333333333333335e-05,
    'lr_outer': 3.3333333333333335e-05,
    'sigmaC': 100.0,
    'batch_size': 1,
}

In [11]:
@dataclass
class Config(BaseConfig):
    n_iter: int = 6001

    n_peers: int = 20
    n_byzan: int = 4

    game_type: GameType = GameType.QUADRATIC
    num_samples: int = 1000
    dim: int = 16
    with_bias: bool = True
    mu: float = 1e-1
    ell: float = 1e3

    attack_type: AttackType = AttackType.ALIE
    n_attacking: int = 1
    ipm_epsilon: float = 1e-1
    rn_sigma: float = 1e1
    alie_z: float = 1e2

    use_bucketing: bool = True
    bucketing_s: int = 2
    aggregator_type: AggregatorType = AggregatorType.RFA
    trimmed_mean_b: int = 10
    krum_m: int = 2
    clipping_tau: int = 10
    clipping_n_iter: int = 3
    rfa_T: int = 10
    rfa_nu: int = 0.1

    optimizer_type: OptimizerType = OptimizerType.SGDARA
    alpha: float = 0.1
    lr: float = 1/2/ell/15
    lr_inner: float = lr
    lr_outer: float = lr
    sigmaC: float = 100.
    batch_size: int = 1

In [12]:
@dataclass
class Data(BaseData):
    def __init__(self, config: BaseConfig):
        self.matrix, self.bias = create_matrix(config.dim, config.num_samples,
                                               config.mu,
                                               config.ell,
                                               with_bias=config.with_bias)
        self.true = torch.linalg.solve(self.matrix.mean(dim=0),
                                       -self.bias.mean(dim=0))
        self.players = self.true + .1/math.sqrt(2 * config.dim)

In [13]:
config = Config()
data = Data(config)

In [14]:
config.__dict__

{'n_iter': 6001,
 'n_peers': 20,
 'n_byzan': 4,
 'game_type': <GameType.QUADRATIC: 'quadratic'>,
 'num_samples': 1000,
 'dim': 16,
 'with_bias': True,
 'mu': 0.1,
 'ell': 1000.0,
 'attack_type': <AttackType.ALIE: 'ALittleIsEnough'>,
 'n_attacking': 1,
 'ipm_epsilon': 0.1,
 'rn_sigma': 10.0,
 'alie_z': 100.0,
 'use_bucketing': True,
 'bucketing_s': 2,
 'aggregator_type': <AggregatorType.RFA: 'RFA'>,
 'trimmed_mean_b': 10,
 'krum_m': 2,
 'clipping_tau': 10,
 'clipping_n_iter': 3,
 'rfa_T': 10,
 'rfa_nu': 0.1,
 'optimizer_type': <OptimizerType.SGDARA: '>'>,
 'alpha': 0.1,
 'lr': 3.3333333333333335e-05,
 'lr_inner': 3.3333333333333335e-05,
 'lr_outer': 3.3333333333333335e-05,
 'sigmaC': 100.0,
 'batch_size': 1}

In [ ]:
config = Config()
config.optimizer_type = OptimizerType.SGDARA
config.aggregator_type = AggregatorType.Mean
config.n_attacking = config.n_byzan
train(BaseConfig(**config.__dict__), BaseData(**data.__dict__))

In [11]:
config = Config()
config.optimizer_type = OptimizerType.SGDARA
config.n_attacking = config.n_byzan
train(BaseConfig(**config.__dict__), BaseData(**data.__dict__))

Trying port 32401


In [12]:
config = Config()
config.optimizer_type = OptimizerType.MSGDARA
config.n_attacking = config.n_byzan
config.alpha = 0.1
train(BaseConfig(**config.__dict__), BaseData(**data.__dict__))

Trying port 31863


In [15]:
config = Config()
config.optimizer_type = OptimizerType.SGDACC
config.n_attacking = 1
train(BaseConfig(**config.__dict__), BaseData(**data.__dict__))

Trying port 12955


In [16]:
config = Config()
config.optimizer_type = OptimizerType.SEGCC
config.n_attacking = 1
config.n_iter = int(config.n_iter / 2) + 1
train(BaseConfig(**config.__dict__), BaseData(**data.__dict__))

Trying port 3995


In [14]:
config = Config()
config.n_iter = int(config.n_iter / 2) + 1
config.n_attacking = config.n_byzan
config.optimizer_type = OptimizerType.RDEG
train(BaseConfig(**config.__dict__), BaseData(**data.__dict__))

Trying port 5969
